<a name="top"> <h1>02. Neuronal Networks</h1> <a>

<p>Análisis de sentimiento: Tweets<br />
<strong>Trabajo de Fin de Master</strong><br />
<strong>Master Universitario en Ciencia de Datos</strong></p>

<p>&nbsp;</p>

<p style="text-align:right">V&iacute;ctor Viloria V&aacute;zquez (<em>victor.viloria@cunef.edu</em>)</p>


<hr style="border:1px solid gray">

### Estructura

[1. Librerias utilizadas y funciones](#librerias) 

[2. Introducción ](#introduccion) 

   - Objetivo de negocio.

[3. Yelp Dataset ](#yelp) 

   - Información del dataset
   - Características del dataset


[4. Transformación del formato de ficheros](#transformacion) 


[5. Transformación de datos](#datos)

   - Business
       - Carga del fichero
       - Transformación de los datos
       - Exportación de ficheros procesados

<hr style="border:1px solid gray">

# <a name="librerias"> 1. Librerias utilizadas y funciones <a>


Importamos las librerias a utilizar para el preprocesamiento:

In [13]:
# Import libraries.

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
import numpy as np
import string

# Import neural network libraries.

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# <a name="lectura"> 2. Lectura del dataframe <a>


In [14]:
#Import parquet file.

reviews = pd.read_parquet('tweets.parquet')

# Show the head of the dataframe.

reviews.head()

,text,sentiment,SentimentText_clean
0,id have responded if i were going,0,id have responded if i were going
1,sooo sad i will miss you here in san diego,2,sooo sad i will miss you here in san diego
2,my boss is bullying me,2,my boss is bullying me
3,what interview leave me alone,2,what interview leave me alone
4,sons of why couldnt they put them on the rel...,2,sons of why couldnt they put them on the rele...


In [15]:
tweets = reviews["SentimentText_clean"].tolist()
sentimientos = reviews["sentiment"].values

In [16]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(tweets)
word_index = tokenizer.word_index

In [18]:
sequences = tokenizer.texts_to_sequences(tweets)


In [19]:
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post", truncating="post")


In [21]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(word_index) + 1, output_dim=64, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation="relu"),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(3, activation="softmax")
])


In [22]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Entrenar el modelo
model.fit(padded_sequences, sentimientos, epochs=10, validation_split=0.2)

Epoch 1/10
687/687 [==============================] - 28s 38ms/step - loss: 0.7909 - accuracy: 0.6372 - val_loss: 0.6646 - val_accuracy: 0.7122
Epoch 2/10
687/687 [==============================] - 6s 8ms/step - loss: 0.5408 - accuracy: 0.7838 - val_loss: 0.6769 - val_accuracy: 0.7237
Epoch 3/10
687/687 [==============================] - 4s 5ms/step - loss: 0.3556 - accuracy: 0.8681 - val_loss: 0.7703 - val_accuracy: 0.7120
Epoch 4/10
687/687 [==============================] - 4s 6ms/step - loss: 0.1922 - accuracy: 0.9359 - val_loss: 0.9607 - val_accuracy: 0.6878
Epoch 5/10
687/687 [==============================] - 4s 6ms/step - loss: 0.0967 - accuracy: 0.9701 - val_loss: 1.1523 - val_accuracy: 0.6791
Epoch 6/10
687/687 [==============================] - 4s 6ms/step - loss: 0.0492 - accuracy: 0.9866 - val_loss: 1.3248 - val_accuracy: 0.6791
Epoch 7/10
687/687 [==============================] - 3s 5ms/step - loss: 0.0316 - accuracy: 0.9916 - val_loss: 1.5085 - val_accuracy: 0.6713
Epoc

In [24]:
# Crear el modelo de la red neuronal
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(word_index) + 1, output_dim=64, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation="relu"),
    tf.keras.layers.MaxPooling1D(5),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv1D(64, 5, activation="relu"),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation="softmax")
])

# Compilar el modelo
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Entrenar el modelo con validación cruzada
history = model.fit(padded_sequences, sentimientos, epochs=20, validation_split=0.2, batch_size=32)



Epoch 1/20
687/687 [==============================] - 22s 29ms/step - loss: 1.0151 - accuracy: 0.4689 - val_loss: 0.7697 - val_accuracy: 0.6631
Epoch 2/20
687/687 [==============================] - 5s 7ms/step - loss: 0.7289 - accuracy: 0.6988 - val_loss: 0.6758 - val_accuracy: 0.7178
Epoch 3/20
687/687 [==============================] - 4s 6ms/step - loss: 0.5802 - accuracy: 0.7719 - val_loss: 0.7030 - val_accuracy: 0.7142
Epoch 4/20
687/687 [==============================] - 4s 7ms/step - loss: 0.4708 - accuracy: 0.8212 - val_loss: 0.7474 - val_accuracy: 0.7033
Epoch 5/20
687/687 [==============================] - 4s 6ms/step - loss: 0.3847 - accuracy: 0.8555 - val_loss: 0.8447 - val_accuracy: 0.7022
Epoch 6/20
687/687 [==============================] - 3s 5ms/step - loss: 0.3166 - accuracy: 0.8860 - val_loss: 0.9472 - val_accuracy: 0.6875
Epoch 7/20
687/687 [==============================] - 5s 7ms/step - loss: 0.2697 - accuracy: 0.9041 - val_loss: 1.0619 - val_accuracy: 0.6789
Epoc

In [26]:
model.save("nn_tweets.h5")